In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import trange

In [2]:
data_test = pd.read_excel("HW3Avalidate.xlsx") 
data_train = pd.read_excel("HW3Atrain.xlsx")

X_train = data_train.copy()
X_train = X_train.drop('y', axis=1)
Y_train = data_train.copy()
Y_train = Y_train.drop('X_0', axis=1)
Y_train = Y_train.drop('X_1', axis=1)

X_test = data_test.copy()
X_test = X_test.drop('y', axis=1)
Y_test = data_test.copy()
Y_test = Y_test.drop('X_0', axis=1)
Y_test = Y_test.drop('X_1', axis=1)


x_0_min = min(X_train['X_0'])
x_0_max = max(X_train['X_0'])

x_1_min = min(X_train['X_1'])
x_1_max = max(X_train['X_1'])

test_x_0_min = min(X_test['X_0'])
test_x_0_max = max(X_test['X_0'])

test_x_1_min = min(X_test['X_1'])
test_x_1_max = max(X_test['X_1'])

for i in range(len(X_train['X_0'])):
    X_train['X_0'][i] = (X_train['X_0'][i]-x_0_min)/(x_0_max-x_0_min) #formula used for Min-Max
    X_train['X_1'][i] = (X_train['X_1'][i]-x_1_min)/(x_1_max-x_1_min) #formula used for Min-Max
    
for i in range(len(X_test['X_0'])):
    X_test['X_0'][i] = (X_test['X_0'][i]-x_0_min)/(x_0_max-x_0_min) #formula used for Min-Max
    X_test['X_1'][i] = (X_test['X_1'][i]-x_1_min)/(x_1_max-x_1_min) #formula used for Min-Max
'''
training_data = []
for i in range(len(X_train['X_0'])):
    training_data.append([round(X_train['X_0'][i], 2),round(X_train['X_1'][i],2)])
    
testing_data = []
for i in range(len(X_test['X_0'])):
    testing_data.append([round(X_test['X_0'][i],2),round(X_test['X_1'][i],2)])

y_train = []

for i in range(len(Y_train['y'])):
    y_train.append([Y_train['y'][i]])
    
y_test = []

for i in range(len(Y_test['y'])):
    y_test.append([Y_test['y'][i]])
'''

training_data = np.empty((0,2), int)
for i in range(len(X_train['X_0'])):
    training_data = np.append(training_data, np.array([[X_train['X_0'][i],X_train['X_1'][i]]]), axis=0)
    
testing_data = np.empty((0,2), int)
for i in range(len(X_test['X_0'])):
    testing_data = np.append(testing_data, np.array([[X_test['X_0'][i],X_test['X_1'][i]]]), axis=0)



y_train = np.empty((0,1), int)
for i in range(len(Y_train['y'])):
    y_train = np.append(y_train, np.array([[Y_train['y'][i]]]), axis=0)

y_test = np.empty((0,1), int)
for i in range(len(Y_test['y'])):
    y_test = np.append(y_test, np.array([[Y_test['y'][i]]]), axis=0)

np_testing_y = np.array(y_test)
np_testing_x = np.array(testing_data)
np_training_x = np.array(training_data)
np_training_y = np.array(y_train)


In [12]:
class Network_layer:
    
    def __init__(self, number_input, number_neurons, use_standard_activation):
        #self.size = size
        self.weights = np.random.rand(number_input,number_neurons)
        self.bias = np.zeros(number_neurons)
        self.last_activation = None
        self.use_standard_activation = use_standard_activation
        self.error = None
        self.delta = None
        
    def activate(self, x):
        r = np.dot(x, self.weights) + self.bias
        if self.use_standard_activation:
            self.last_activation = self.activation(r)
        else: 
            self.last_activation = self.tanh(r)
            
        return self.last_activation
    
    def activation(self, x):
        #relu
        x[x<0] = 0
        return x
                
    def tanh(self, x):
        #tanh
        return np.tanh(x)
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def activation_derivative(self, x):
        if self.use_standard_activation:
            x[x>0] = 1
            x[x<0] = 0
            return x
        return 1.0 - np.tanh(x)**2
        '''
        if self.use_standard_activation:
            return 1.0 - np.tanh(x)**2
        return x * (1 - x)
        '''
    

class Neaural_net:
    layers = []
    
    def __init__(self):
        self.layers = []
    
    def add_layer(self, layer):
        self.layers.append(layer)
    
    def feed_forward(self, X):
        for layer in self.layers:
            X = layer.activate(X)
        return X

        
    def backpropagation(self, X, y, learning_rate):
        #print(y)
        #print(X)
        output = self.feed_forward(X)
        #for i in reversed(range(len(self.layers))):
        #    print(self.layers[i].weights.shape)

        #for j in range(len(output)):
        for i in reversed(range(len(self.layers))):
            #print(y[0])
            layer = self.layers[i]
            if layer == self.layers[-1]:
                layer.error = y - output
                print(layer.error)
                #layer.error = np.square(y - NN.feed_forward(X))
                layer.delta = layer.error * layer.activation_derivative(output)
                #print(layer.delta)
            else:
                next_layer = self.layers[i + 1]
                layer.error = np.dot(next_layer.weights, next_layer.delta)
                layer.delta = layer.error * layer.activation_derivative(layer.last_activation)
                #print('sdf', layer.delta)


        # Update the weights
        for i in range(len(self.layers)):
            layer = self.layers[i]
            input_to_use = np.atleast_2d(X if i == 0 else self.layers[i - 1].last_activation)
            layer.weights += layer.delta * input_to_use.T * learning_rate
            
    
    
    def iterate_minibatches(self, inputs, targets, batchsize, shuffle=False):
        assert len(inputs) == len(targets)
        if shuffle:
            indices = np.random.permutation(len(inputs))
        for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
            if shuffle:
                excerpt = indices[start_idx:start_idx + batchsize]
            else:
                excerpt = slice(start_idx, start_idx + batchsize)
            yield inputs[excerpt], targets[excerpt]

    '''
    from IPython.display import clear_output
    train_log = []
    val_log = []
    for epoch in range(25):
        for x_batch,y_batch in iterate_minibatches(X_train,y_train,batchsize=5,shuffle=True):
            train(network,x_batch,y_batch)
        train_log.append(np.mean(predict(network,X_train)==y_train))
        val_log.append(np.mean(predict(network,X_test)==y_test))
    '''    
            
    def train(self, X, y, learning_rate, max_epochs):
        mses = []

        for i in range(max_epochs):
            #for x_batch,y_batch in self.iterate_minibatches(X,y,5,shuffle=True):

             #   self.backpropagation(x_batch,y_batch, learning_rate)
            for j in range(len(X)):
                self.backpropagation(X[j], y[j], learning_rate)
            if i % 100 == 0:
                mse = np.mean(np.square(y - NN.feed_forward(X)))
                mses.append(mse*100)
        return mses
    
    
    def test(self, X, y_true):
        correct = 0
        
        for i in range(len(X)):
            forward_pass = self.feed_forward(X[i])
            if np.around(forward_pass) == y_true[i]:
                correct = correct + 1
        return correct/len(X)



NN = Neaural_net()
NN.add_layer(Network_layer(2, 10, False))
NN.add_layer(Network_layer(10, 10, False))
NN.add_layer(Network_layer(10, 10, False))
NN.add_layer(Network_layer(10,1, False))


# Train the neural networks
accuracy = []
iteration = []
learning_rate = 0.01
total_error = []
number_test = 100
number_epoc = 20
last_mse = [0,0,0,0,0]
a = 0


    
    

for i in range(1,number_test+1):
    errors = NN.train(np_training_x, np_training_y, learning_rate, number_epoc)
    total_error.append(errors)
    learning_rate = learning_rate*0.99
    last_mse[a] = round(errors[0],2)
    print(last_mse[a])
    a = a + 1
    if a > 4:
        a = 0
    iteration.append(i*number_epoc)
    accuracy_one = NN.test(np_testing_x, np_testing_y.flatten())
    print('Accuracy-iteration: %.2f%%' % (accuracy_one*100), "Learning rate --> "+ str(learning_rate))
    accuracy.append(accuracy_one*100)
    mse_same = True
    for i in range(len(last_mse)):
        if round(last_mse[i],2) != round(errors[0],2):
            mse_same = False
    if mse_same:
        print("MSE has converged!")
        break;


plt.plot(iteration,accuracy)
plt.plot(iteration, total_error)
plt.xlabel('iterations')
plt.ylabel('Accuracy & MSE(*100)')
plt.axis((number_epoc,number_epoc*number_test,0,100))
plt.title('Changes in Accuracy')
plt.show()

data = {'weights': []}
data['weights'].append(NN.layers[0].weights.tolist())
data['weights'].append(NN.layers[1].weights.tolist())
data['weights'].append(NN.layers[2].weights.tolist())
with open('data.txt', 'w') as outfile:
    json.dump(data, outfile)
    


[-0.99998142]
[-0.99997747]
[-0.999978]
[-0.99997625]
[-0.99997416]
[-0.9999711]
[3.06190455e-05]
[-0.99996943]
[-0.999964]
[3.6225575e-05]
[3.61632558e-05]
[-0.99996388]
[-0.99996071]
[4.27474094e-05]
[-0.99995726]
[-0.99995243]
[5.06072973e-05]
[5.06478165e-05]
[5.05748202e-05]
[-0.99994934]
[-0.9999441]
[5.98436403e-05]
[6.0501387e-05]
[-0.9999395]
[-0.99993075]
[-0.99992931]
[-0.99992313]
[-0.99987496]
[9.08377074e-05]
[9.08913517e-05]
[-0.99990936]
[9.88213183e-05]
[9.9015461e-05]
[-0.99990144]
[0.00010752]
[-0.99989279]
[0.0001172]
[-0.99988241]
[0.00012765]
[-0.9998719]
[0.00013855]
[-0.99986194]
[0.00015073]
[0.00015074]
[0.00015009]
[0.00015081]
[0.00015016]
[0.00015042]
[-0.99984266]
[0.0001635]
[-0.99983589]
[-0.99982274]
[-0.99829635]
[0.00020571]
[-0.99979427]
[0.0002241]
[0.00022428]
[0.00022584]
[0.00022499]
[0.00022658]
[-0.99977622]
[-0.99975505]
[-0.99973557]
[0.00028791]
[0.00028868]
[-0.99969117]
[0.00031467]
[-0.99936663]
[0.00033897]
[0.0003389]
[0.00034074]
[-0.9

[0.70311077]
[-0.39014709]
[-0.29608452]
[0.68410017]
[0.62387478]
[0.58901152]
[0.59586209]
[0.55265442]
[-0.51373835]
[0.51557765]
[-0.50868488]
[0.51422399]
[-0.50955578]
[-0.44463537]
[0.6108891]
[-0.49557791]
[0.5796672]
[0.54679181]
[-0.48841486]
[-0.4380673]
[-0.10005163]
[0.51798942]
[-0.36237534]
[-0.49498252]
[0.58685006]
[-0.49801106]
[-0.43799505]
[-0.45012276]
[0.59756676]
[0.54411045]
[0.54468011]
[0.51638518]
[0.51080934]
[0.48439617]
[0.42915615]
[-0.5912933]
[-0.56913981]
[-0.54131889]
[-0.517067]
[0.53943355]
[0.5096944]
[-0.37703581]
[-0.52331915]
[0.51059303]
[0.49183586]
[-0.26029786]
[-0.44402807]
[-0.52112555]
[0.4950971]
[-0.02507045]
[-0.44571928]
[-0.48027699]
[0.53627609]
[0.50248225]
[-0.37280927]
[-0.51471452]
[-0.43187124]
[0.52269776]
[-0.31406567]
[0.53860956]
[-0.42010383]
[-0.29387207]
[-0.40273747]
[-0.46289617]
[0.57158881]
[0.52843411]
[0.54277274]
[-0.51965214]
[0.56465619]
[-0.52754108]
[-0.49519697]
[0.55136248]
[-0.49764578]
[0.52595995]
[0.5011

[0.36643545]
[0.35503388]
[0.32721245]
[-0.57686654]
[-0.58085498]
[-0.63981402]
[0.40466712]
[-0.6275423]
[-0.59922384]
[-0.33318467]
[-0.55882838]
[0.47187922]
[-0.55357947]
[-0.51656995]
[0.50403417]
[-0.51452917]
[-0.33908981]
[-0.4245202]
[-0.35652615]
[-0.22301593]
[0.57741106]
[-0.41616352]
[-0.37728019]
[-0.35359947]
[0.63907175]
[0.5473046]
[-0.4200919]
[-0.44162161]
[-0.39929663]
[-0.36994213]
[0.64213391]
[-0.40898712]
[-0.3774697]
[-0.34490704]
[-0.18055855]
[0.66658932]
[-0.35086581]
[0.62008482]
[-0.22098186]
[0.62567824]
[0.54280124]
[-0.42568771]
[-0.36975313]
[-0.37232458]
[-0.23029503]
[-0.29618357]
[0.75058507]
[-0.35648643]
[-0.30151187]
[0.75735458]
[0.62062242]
[0.56870199]
[0.62730635]
[0.57590301]
[-0.50458848]
[0.49621153]
[-0.48832694]
[0.52141753]
[-0.50147299]
[-0.44932813]
[0.6419146]
[-0.48559976]
[0.59162498]
[0.55372224]
[-0.49007402]
[-0.43327893]
[-0.0920384]
[0.51110635]
[-0.34445276]
[-0.47751759]
[0.60619208]
[-0.48111641]
[-0.42736099]
[-0.40865204

[-0.4718847]
[-0.32890221]
[-0.2920951]
[-0.42832907]
[0.5165752]
[0.64061758]
[-0.50116165]
[0.60613867]
[-0.31387667]
[0.5411266]
[-0.51408279]
[0.455756]
[0.53235538]
[0.40537598]
[-0.56543954]
[0.53449524]
[0.46144259]
[-0.04979977]
[0.39317305]
[0.41529001]
[0.37663676]
[0.32994993]
[0.35476899]
[-0.40752225]
[-0.28498787]
[-0.67834444]
[-0.61552954]
[-0.58518321]
[-0.56105676]
[-0.5052548]
[-0.30906741]
[0.57199613]
[-0.26883978]
[-0.24375747]
[-0.46143114]
[0.58398386]
[-0.48856724]
[0.55746781]
[0.51469817]
[-0.5357512]
[0.50225614]
[0.45468267]
[-0.55397514]
[0.49953851]
[0.45004097]
[0.41846118]
[0.41326542]
[-0.48546556]
[0.39998202]
[0.39111238]
[0.35848479]
[-0.35142425]
[-0.56323315]
[0.3786557]
[-0.58633892]
[-0.3434394]
[-0.54146491]
[-0.55193986]
[-0.52698756]
[-0.37079841]
[0.5120885]
[-0.51282421]
[-0.28862021]
[0.53027923]
[0.50521538]
[-0.49811427]
[-0.42398472]
[0.56901792]
[-0.48003572]
[-0.36665359]
[0.55035244]
[0.5263475]
[0.48020624]
[-0.55753544]
[-0.4417355

[-0.5644932]
[0.46516791]
[0.34080686]
[-0.5568229]
[0.45238431]
[0.41921607]
[0.38511433]
[-0.58160901]
[0.34424578]
[-0.62881]
[-0.56987212]
[0.38810778]
[0.32758068]
[0.29333987]
[0.34138461]
[-0.68840447]
[-0.61869432]
[-0.50799193]
[0.37494279]
[0.42962734]
[-0.39631592]
[-0.50556864]
[0.4182594]
[0.38227191]
[0.30245304]
[-0.62652369]
[0.40788699]
[0.36131603]
[-0.59917711]
[-0.55505745]
[-0.47263464]
[0.53356419]
[0.45604832]
[-0.4306078]
[-0.49347766]
[-0.28405213]
[0.48659206]
[0.38567279]
[-0.44751969]
[0.44704643]
[0.41997061]
[0.36957074]
[-0.51551865]
[-0.55776536]
[-0.49075118]
[0.46572198]
[-0.36166592]
[0.41636893]
[-0.44133999]
[0.57408474]
[0.48271049]
[-0.5605139]
[0.40759463]
[-0.21129313]
[0.43418627]
[-0.50622162]
[-0.4468559]
[-0.42903595]
[-0.34627587]
[0.48973958]
[0.49604948]
[0.39984578]
[0.55139923]
[-0.55514943]
[-0.56240697]
[-0.38163925]
[0.44686766]
[-0.34326097]
[-0.34445534]
[-0.31649602]
[-0.34350813]
[0.50777325]
[0.70755878]
[-0.4536963]
[0.6013493]

...

[0.11528482]
[-0.03540364]
[-0.20235863]
[-0.16748468]
[-0.22505386]
[0.29690272]
[0.66823812]
[-0.33519471]
[-0.55626212]
[0.29171171]
[0.58587689]
[0.24089654]
[0.29357483]
[0.17423859]
[-0.4361689]
[0.20335575]
[-0.41232678]
[0.80399985]
[-0.57557481]
[-0.62109887]
[0.37487262]
[-0.24588663]
[0.56152937]
[0.41674571]
[-0.61540097]
[-0.48040179]
[-0.09553059]
[0.32707417]
[-0.37352382]
[0.07048101]
[0.24937654]
[-0.09593233]
[-0.68684819]
[0.30515451]
[0.44567724]
[0.39604055]
[0.1437505]
[0.35672383]
[0.17109588]
[0.12031896]
[0.06515125]
[-0.63095536]
[-0.32841147]
[-0.38148651]
[0.07782184]
[0.64859185]
[0.17131706]
[-0.3987074]
[-0.3713023]
[0.45273227]
[0.36303994]
[-0.30555604]
[-0.60922722]
[-0.29066849]
[0.49503125]
[-0.03175204]
[-0.4758711]
[-0.42538668]
[0.81806167]
[0.25791108]
[-0.39874791]
[-0.51737791]
[-0.59679543]
[0.42021318]
[-0.2164686]
[0.53435192]
[-0.50872991]
[-0.36501417]
[-0.6246927]
[-0.05776585]
[0.65793099]
[0.22926322]
[0.16818754]
[-0.46811223]
[0.27258

[0.24420006]
[-0.40231673]
[0.46583293]
[0.10613015]
[0.44022438]
[0.33127292]
[-0.56127505]
[0.15435314]
[0.14957469]
[0.39864608]
[-0.42001479]
[-0.63962413]
[0.35472244]
23.9
Accuracy-iteration: 79.27% Learning rate --> 0.0099
[-0.58926715]
[-0.35737716]
[-0.57074849]
[0.12338509]
[-0.11427537]
[-0.4799768]
[0.47519527]
[-0.30364101]
[-0.22142694]
[0.38344639]
[0.31190063]
[0.09155313]
[0.14543912]
[0.67521276]
[-0.62788601]
[-0.52253514]
[0.29937202]
[0.14006024]
[0.14168025]
[-0.88455321]
[-0.47721708]
[0.09799075]
[0.37933343]
[-0.52956701]
[-0.3172665]
[-0.04246647]
[-0.00763665]
[-0.16520154]
[0.10699393]
[0.56679698]
[-0.24543138]
[0.13439302]
[0.31169968]
[-0.21036038]
[0.39874317]
[-0.23127022]
[0.3317967]
[-0.45888875]
[0.38590944]
[-0.47628981]
[0.59146374]
[-0.3195428]
[0.11799366]
[0.51751022]
[0.32739015]
[0.35892244]
[0.18775623]
[0.07143738]
[-0.56633961]
[0.1974357]
[-0.64941295]
[-0.02972391]
[-0.12861803]
[0.24983093]
[-0.45549132]
[0.63544786]
[0.18028107]
[0.2485

[0.21689854]
[-0.62836568]
[-0.46722743]
[0.38525929]
[0.16655065]
[0.12141413]
[0.12842882]
[-0.38085525]
[-0.05384744]
[-0.53351593]
[0.21097904]
[0.37027444]
[-0.27899822]
[-0.38815218]
[0.36154447]
[0.47162758]
[0.10012288]
[-0.25567261]
[0.40192419]
[0.30290617]
[-0.48825605]
[0.00166518]
[-0.09843332]
[0.51000441]
[0.24784211]
[-0.33866117]
[-0.13164965]
[-0.289511]
[0.55694991]
[0.08576759]
[-0.13331989]
[0.30901603]
[0.37825883]
[0.15243251]
[-0.55464443]
[-0.62685574]
[-0.21653468]
[0.46938861]
[-0.07155325]
[0.12134924]
[-0.39862218]
[0.11329796]
[0.23141856]
[-0.2950147]
[0.19263145]
[-0.12494003]
[0.324056]
[-0.33106042]
[-0.16068569]
[-0.17528481]
[0.22089799]
[0.17566838]
[0.27180273]
[0.20918335]
[0.22824233]
[-0.43361454]
[-0.52490001]
[-0.1789147]
[0.32146171]
[0.28681841]
[-0.22937506]
[-0.28279864]
[-0.07630048]
[0.26723824]
[0.20810447]
[-0.18338613]
[0.57703647]
[-0.42724335]
[0.53700861]
[-0.47436197]
[0.16912455]
[0.16723128]
[0.24888842]
[-0.25473243]
[0.0768913

[0.13443418]
[0.04801065]
[0.03286212]
[-0.5216229]
[-0.16788205]
[-0.27375647]
[0.10122681]
[0.70898014]
[0.05337133]
[-0.26715914]
[-0.39233463]
[0.38447303]
[0.1899616]
[-0.19553291]
[-0.55708142]
[-0.12512057]
[0.23079226]
[-0.0195433]
[-0.21211287]
[-0.37246186]
[1.04991146]
[0.17876667]
[-0.34906874]
[-0.51491712]
[-0.59117477]
[0.19063509]
[-0.02912991]
[0.34502926]
[-0.27538245]
[-0.21919233]
[-0.58641629]
[-0.03271098]
[0.43125115]
[0.17489991]
[0.13736062]
[-0.15242247]
[0.1137537]
[0.1152818]
[-0.28975633]
[0.2066111]
[0.05377257]
[1.0387369]
[0.04444264]
[-0.34327053]
[-0.43496445]
[0.14211182]
[0.25107117]
[0.06421575]
[0.15205772]
[-0.50983786]
[-0.38751674]
[0.04677778]
[-0.36052368]
[0.03504911]
[0.0390842]
[0.06129068]
[0.95179627]
[-0.51912314]
[0.04538677]
[0.42029607]
[0.14515167]
[-0.7682155]
[0.17294517]
[0.35662118]
[0.05938902]
[-0.38634712]
[0.10146973]
[0.22620942]
[-0.18019452]
[0.39678635]
[0.04389102]
[0.09044175]
[-0.67224969]
[0.21392878]
[-0.59222553]
[-

KeyboardInterrupt: 

## 1. (10 points) Activation and Loss functions. 
Please choose suitable activation functions φ(0), φ(1), φ(2) and a suitable Loss function to perform the task. Report and justify your choices in the report.

## 2. (10 points) Learning rate, batch size, initialization. 
Please choose a suitable learning rate, batch size, initialization of the parameter values, and any other setting you may need. Discuss and justify your choices in the report.

## 3. (10 points) Training. 
Make plots with the loss function computed over the training set and over the validation set. Stop the training when the error is small enough. Justify your stopping criterium. Report the final accuracy obtained and the confusion matrix on the validation dataset.

## 4. (20 points) Implementation. 
We will run and check the uploaded Python file. To obtain the points for this subproblem, the Python file has to run (no errors) and the MLP model and the Backpropagation algorithm have to be implemented completely from scratch by you. You are not allowed to use any library which implements MLP models, but you are allowed to use auxiliary libraries, e.g. Numpy, Matplotlib, Pandas.

## II. Peer Review paragraph (0 points)
Finally, each group member must write a single paragraph outlining their opinion on the work distri- bution within the group. Did every group member contribute equally? Did you split up tasks in a fair manner, or jointly worked through the exercises. Do you think that some members of your group deserve a different grade from others?